In [9]:
import numpy as np
from lat_type import lat_type
from phenum.vector_utils import _minkowski_reduce_basis
from phenum.symmetry import get_lattice_pointGroup
from base_ortho import base_ortho_srHNFs as srHNFs
from stet import stet_srHNFs as stSrHNFs
from copy import deepcopy
#import niggli

Lets first assume two basis that are simply the rotations of each other, A=[[1,0],[0,2]] and B = [[2,0],[0,1]]. These two are equivalent by a 90 degree rotation.

We need to check that we can map from one to the other properly.

The issue seems to at least partially be the order of the vectors [a,b,c] if I can find a way to ensure that the users ordering matches ours then I think we have a solution.

Procedure should be to minkowski reduce the users basis, find the order of the vectors for our basis, then get the user's basis into the same order. Once that is accomplished we can find the mapping from th user's basis to ours and proceed per normal.


In [2]:
def get_A(A_m,order_c):
    o_norms = list(np.linalg.norm(A_m,axis=0))
    s_norms = sorted(o_norms)

    order = []
    for i in range(3):
        temp = s_norms[order_c[i]]
        
        for j in range(3):
            if np.allclose(o_norms[j],temp) and j not in order:
                order.append(j)
                break

    A = np.transpose(np.transpose(A_m)[order])
    return A

In [3]:
    As = [[[-1,1,1],[1,-1,1],[1,1,-1]],[[1,0,0],[0,1,0],[0.5,0.5,0.5]],
          [[1,0,0],[0.5,0.5,0.5],[0,0,1]],[[0.5,-0.5,0.5],[0,1,0],[0,0,1]],
          [[0,0,2],[1,-1,1],[1,1,-1]],[[-1,1,1],[2,0,0],[0,2,0]],
          [[1,-1,3],[1,-1,1],[1,1,-1]]]

As = [np.transpose(i) for i in As]

In [4]:
k= 6
Rp = [[0.5,-0.866025404,0],[0.866025404,0.5,0],[0,0,1]]
eps = 1E-10
A_t = np.transpose([[0.5,0.5,-0.5],[-0.5,0.5,0.5],[1,0,1]])#np.dot(Rp,np.random.random()*As[k])
A_m = np.transpose(_minkowski_reduce_basis(np.transpose(A_t),1E-10))
lat_id, B, order = lat_type(np.transpose(A_t),eps=eps)
#B = np.transpose(np.array([[-1,1,1],[1,-1,1],[1,1,-1]]))
#lat_id = 'bo'
#order = [0,1,2]
B_m = np.transpose(_minkowski_reduce_basis(np.transpose(B),1E-10))
A_m = get_A(A_m,order)
B_m = get_A(B_m,order)
A = deepcopy(A_t)
A_s = get_A(A_t,order)
HNFs = srHNFs(26)
#print(len(HNFs))
#lat_id

In [5]:
B

array([[ 0.5,  0.5,  0. ],
       [ 1. , -1. ,  0. ],
       [ 0. ,  0. ,  3. ]])

In [14]:
print(np.dot(np.transpose(B),B))
np.dot(np.transpose(niggli.niggli_reduce(B)),niggli.niggli_reduce(B))

[[ 1.25 -0.75  0.  ]
 [-0.75  1.25  0.  ]
 [ 0.    0.    9.  ]]


array([[ 1.  , -0.5 ,  0.  ],
       [-0.5 ,  1.25,  0.  ],
       [ 0.  ,  0.  ,  9.  ]])

In [11]:
print(np.dot(np.transpose(A),A))
niggli.niggli_reduce(A)

[[ 0.75 -0.25  0.  ]
 [-0.25  0.75  0.  ]
 [ 0.    0.    2.  ]]


array([[ 0.5, -0.5,  1. ],
       [ 0.5,  0.5,  0. ],
       [-0.5,  0.5,  1. ]])

In [6]:
def is_orthonormal(matrix,eps=1E-6):
    """Determines if the input matrix is orthonormal.
    
    Args:
        matrix (numpy array): The 3 by 3 matrix to check.
        eps (float optional): Floating point tolerance, 
            default is 1E-6.
    Returns:
        ortho (bool): True if matrix is orthonormal.
    """
    
    matrix = np.array(matrix)
    
    ortho = True
    vals = list(range(3))
    for i in vals:
        prod = abs(np.dot(matrix[:,i],matrix[:,i]))
        if abs(prod-1.0) > eps:
            print(prod)
            print("F1")
            ortho = False
            break
        for j in vals:
            if i != j:
                prod2 = abs(np.dot(matrix[:,i],matrix[:,j]))
                if prod2 > eps:
                    print("F2")
                    ortho = False 
                    break
    return ortho        

In [101]:
np.linalg.norm(B_m,axis=0)

array([ 1.        ,  1.11803399,  3.        ])

In [111]:
Sc

array([[ 0.8660254 ,  0.        ,  0.        ],
       [ 0.        ,  0.77459667,  0.        ],
       [ 0.        ,  0.        ,  0.47140452]])

In [114]:
Sc = np.identity(3)*np.transpose(np.linalg.norm(A_m,axis=0)/np.linalg.norm(B_m,axis=0))
permutations = [[0,1,2],[1,0,2],[1,2,0],[0,2,1],[2,0,1],[2,1,0]]
correct = False
current = 0
while not correct:
    perm = permutations[current]
    R = np.dot(np.transpose(np.transpose(A_m)[perm]),np.linalg.inv(np.dot(B_m,Sc)))
    print(R,np.linalg.det(R))
    if is_orthonormal(R,eps=1E-1):
        correct = True
    else:
        current += 1
        
    if current == 6:
        print("H")
        R = np.array([[1,0,0],[0,1,0],[0,0,1]])
        correct = True

R = np.dot(np.linalg.inv(B_m),A_m)        
L = np.dot(np.linalg.inv(A_s),np.dot(R,B))
#print(np.transpose(A_m))
#print(np.transpose(np.dot(Sc,B_m)))
#print(np.linalg.det(R))
#print(R)

(array([[ 0.57735027, -0.35682209,  0.70710678],
       [ 0.57735027,  0.93417236,  0.        ],
       [-0.57735027,  0.35682209,  0.70710678]]), 1.0540925533894601)
F2
1.12732200375
F1
(array([[-0.57735027,  0.35682209,  0.70710678],
       [ 0.57735027,  0.93417236,  0.        ],
       [ 0.57735027, -0.35682209,  0.70710678]]), -1.0540925533894601)
F2
1.12732200375
F1
(array([[-0.57735027,  1.00231931,  0.35355339],
       [ 0.57735027,  0.28867513,  0.35355339],
       [ 0.57735027,  1.57966958, -0.35355339]]), 1.0540925533894601)
F2
3.58333333333
F1
(array([[ 0.57735027,  1.57966958, -0.35355339],
       [ 0.57735027,  0.28867513,  0.35355339],
       [-0.57735027,  1.00231931,  0.35355339]]), -1.0540925533894601)
F2
3.58333333333
F1
(array([[ 1.15470054,  1.22284749, -0.35355339],
       [ 0.        ,  0.64549722,  0.35355339],
       [ 1.15470054, -0.06814696,  0.35355339]]), 1.0540925533894601)
2.66666666667
F1
(array([[ 1.15470054, -0.06814696,  0.35355339],
       [ 0.      

In [160]:
trans = []
for k in range(3):
    temp = np.outer(B[:,k],A_t[:,k])/np.dot(B[:,k],B[:,k])
    trans.append(temp)

In [10]:
S = np.dot(B,HNFs[5])
#print(lat_type(np.transpose(S),eps=eps)[0])
S_a = []
for k in range(3):
    S_a.append(np.dot(S[:,k],trans[k]))
S_a = np.transpose(S_a)
#S_a=np.dot(R,np.dot(S,np.linalg.inv(L)))
#print(lat_type(np.transpose(S_a),eps=eps)[0])

NameError: name 'trans' is not defined

In [11]:
S

array([[  6.5,   6.5,   0. ],
       [-11. , -13. ,   0. ],
       [  0. ,   0. ,   6. ]])

In [173]:
g = np.transpose(np.linalg.inv(S_a))
r = np.transpose(np.linalg.inv(A_t))
#print(lat_type(np.transpose(g),eps=eps)[0])
np.round(np.dot(np.linalg.inv(g),r),3)

array([[ -6.2,   0. ,   0. ],
       [  0. ,  13. ,   0. ],
       [  0. ,   0. ,   2. ]])

In [82]:
A_m

array([[ 0.5, -0.5,  1. ],
       [ 0.5,  0.5,  0. ],
       [-0.5,  0.5,  1. ]])

In [83]:
A_s

array([[ 0.5, -0.5,  1. ],
       [ 0.5,  0.5,  0. ],
       [-0.5,  0.5,  1. ]])

In [84]:
np.linalg.norm(A_s,axis=0)

array([ 0.8660254 ,  0.8660254 ,  1.41421356])

In [18]:
A_t

array([[ 0.5, -0.5,  1. ],
       [ 0.5,  0.5,  0. ],
       [-0.5,  0.5,  1. ]])

In [119]:
B

array([[ 0.5,  0.5,  0. ],
       [ 1. , -1. ,  0. ],
       [ 0. ,  0. ,  3. ]])

In [120]:
S = np.dot(A_t,np.linalg.inv(B))

In [121]:
T = np.linalg.inv(S)

In [122]:
T

array([[ 0. ,  1. ,  0. ],
       [ 1. ,  0. , -1. ],
       [ 1.5,  0. ,  1.5]])

In [150]:
Bs = np.dot(B,HNFs[2])

In [151]:
As = np.dot(S,Bs)

In [154]:
lat_type(np.transpose(g))

('so', array([[1, 0, 0],
        [0, 2, 0],
        [0, 0, 3]]), [0, 1, 2])

In [153]:
r = np.linalg.inv(np.transpose(A_t))
g = np.linalg.inv(np.transpose(As))
np.round(np.dot(np.linalg.inv(g),r),3)

array([[  1.,   1.,   0.],
       [  0.,   2.,   0.],
       [  0.,   0.,  13.]])

In [25]:
from phenum import symmetry

In [20]:
lat_type([[0.06666666666667, -0.03849001811417425, -0.10206207023193052], 
          [0., -0.0769800362283485, 0.10206207023193052], 
          [-0.06666666666667, -0.03849001811417425, -0.10206207023193052]],eps=1E-10)

('cm', array([[ 1. ,  0. ,  0.5],
        [ 1. ,  2. ,  0. ],
        [ 0. ,  0. ,  2. ]]), [0, 1, 2])

In [37]:
len(symmetry.get_lattice_pointGroup([[0.08333333333333, -0.048112522642713484, -0.040824828092772214],
                                     [0., -0.09622504528542697, 0.040824828092772214], 
                                     [-0.08333333333333, -0.048112522642713484, -0.040824828092772214]],eps=1E-3))

12

In [1]:
from pyniggli import reduced_cell
from niggli import niggli_reduce
import numpy as np

In [2]:
a = [1,-.1,0]
b = [-0.3,1,.3]
c = [-.3,-0.1,-1.5 ]
print(np.dot(a,a),np.dot(b,b),np.dot(c,c),2.0*np.dot(b,c), 2.0*np.dot(c,a), 2.0*np.dot(a,b))

(1.01, 1.1800000000000002, 2.3500000000000001, -0.91999999999999993, -0.57999999999999996, -0.80000000000000004)


In [8]:
A = np.transpose([[0.5,0,0.5],[0,3,0]])
B = reduced_cell(A)

ValueError: The input basis must be a 3 by 3 matrix with the lattice vectors as columns of the matrix.

In [7]:
print(np.transpose(B.niggli))
niggli_reduce(A)

[[-0.5  0.  -0.5]
 [-0.5  0.   0.5]
 [ 0.  -3.   0. ]]


array([[-0.5, -0.5,  0. ],
       [ 0. ,  0. , -3. ],
       [-0.5,  0.5,  0. ]])

In [10]:
B.C

array([[ 0.,  1.,  0.],
       [ 1.,  2.,  1.],
       [-1., -1., -2.]])

In [48]:
temp = np.transpose(niggli_reduce(A))
a = np.dot(temp[0],temp[0])
b=np.dot(temp[1],temp[1])
c=np.dot(temp[2],temp[2])
xi=2.0*np.dot(temp[1],temp[2])
eta=2.0*np.dot(temp[2],temp[0])
zeta=2.0*np.dot(temp[0],temp[1])
print("a",a,"b",b,"c",c,"xi",xi,"eta",eta,"zeta",zeta)

('a', 1.0, 'b', 5.6666667650268803, 'c', 17.666667105991401, 'xi', -1.6666672436430421, 'eta', -1.0, 'zeta', 0.0)


In [24]:
temp

array([[-0.89,  0.4 , -0.9 ],
       [-0.84, -1.6 ,  0.3 ],
       [-1.68,  1.5 ,  2.7 ]])

In [5]:
temp = [[1.00000000000000, 0.00000000000000,0.00000000000000],
                     [0.50000000000000,0.86602540000000,1.63299320000000],
                     [0.00000000000000,-1.73205080000000,1.63299320000000]]
a = np.dot(temp[0],temp[0])
b=np.dot(temp[1],temp[1])
c=np.dot(temp[2],temp[2])
xi=2.0*np.dot(temp[1],temp[2])
eta=2.0*np.dot(temp[2],temp[0])
zeta=2.0*np.dot(temp[0],temp[1])
print("a",a,"b",b,"c",c,"xi",xi,"eta",eta,"zeta",zeta)

('a', 1.0, 'b', 3.6666667846914001, 'c', 5.6666667650268803, 'xi', 2.333333608711841, 'eta', 0.0, 'zeta', 1.0)


In [22]:
A = np.transpose([[0,.5,.5],[.5,0,.5],[.5,.5,0]])
B = reduced_cell(A)
np.transpose(B.niggli)

array([[ 0. ,  0.5,  0.5],
       [ 0.5,  0. ,  0.5],
       [ 0.5,  0.5,  0. ]])

In [23]:
a = B.niggli[:,0]
b = B.niggli[:,1]
c = B.niggli[:,2]

In [24]:
print(np.dot(b,c),np.dot(a,c),np.dot(a,b))

(0.25, 0.25, 0.25)


In [25]:
np.dot(a,a)/2.

0.25

In [11]:
import numpy as np
from phenum.symmetry import get_lattice_pointGroup
from opf_python.stet import stet_11

In [16]:
At = [[-4.85960351639766, 4.86010400245928, 2.37204283593526],
      [4.86010400245928, -4.85960351639766, 2.37204283593526],
      [4.84101040880108, 4.84101040880108, -2.37101750239167]]
A = np.transpose(At)

In [17]:
pg = get_lattice_pointGroup(At, eps=1E-3)
print(len(pg))

4


In [21]:
pgi = [np.round(np.matmul(np.linalg.inv(A),np.matmul(g, A))) for g in pg]
pgi

[array([[-1.,  0.,  0.],
        [ 0., -1.,  0.],
        [-0.,  0., -1.]]), array([[ 0., -1.,  0.],
        [-1.,  0.,  0.],
        [ 0., -0., -1.]]), array([[-0.,  1., -0.],
        [ 1.,  0., -0.],
        [-0.,  0.,  1.]]), array([[ 1., -0., -0.],
        [ 0.,  1., -0.],
        [ 0., -0.,  1.]])]

In [13]:
hnfs = stet_11(1000)
print(len(hnfs))

14


In [51]:
K = np.linalg.inv(np.transpose(np.matmul(A, hnfs[13])))
K

array([[  1.75244756e-05,   5.14418771e-03,   2.06557774e-02],
       [  1.03058999e-02,  -5.14418771e-03,   2.06557774e-02],
       [  2.10777883e-02,   1.42828649e-22,  -4.35823861e-06]])

In [52]:
pki = [np.round(np.matmul(np.linalg.inv(K),np.matmul(g, K)),3) for g in pg]
pki

[array([[-1.,  0.,  0.],
        [-0., -1.,  0.],
        [-0.,  0., -1.]]), array([[-1., -0.,  0.],
        [-2.,  1., -0.],
        [-0., -0., -1.]]), array([[ 1.,  0., -0.],
        [ 2., -1.,  0.],
        [ 0.,  0.,  1.]]), array([[ 1., -0., -0.],
        [ 0.,  1.,  0.],
        [ 0., -0.,  1.]])]

In [ ]:
import phenum